In [ ]:
# Make plots to visualize data collected by ChorusWaves search script
# Date created: not sure sorry :(
# Last modified: 10/24/24
# Author: Max Feinland for Blum Research Group

In [ ]:
'''
Example event plots
'''

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sampex # thanks to Mike Shumko for making this package
import matplotlib.dates as dates
from scipy.signal import find_peaks

plt.rcParams["font.family"] = "Arial" # bc I don't like the default Python font

def plot_date(ax, j):
    # Function that makes plots of events
    # Author: Max Feinland
    # Date created: not sure, sorry
    # Last modified: 7/23/2024
    
    # Inputs: axes object (which subplot to plot on), iterator variable, 
    # dictionary containing specifics for each plot
    
    # Outputs: none
    
    b1 = 40 # buffer between start time and start of plotting (in indices)
    b2 = 150 # buffer between stop time and end of plotting (in indices)
    t = plotting_data["t"][j] # timestamp of event start
    
    cur_t = pd.to_datetime(t) # convert text to DateTime
    h = sampex.HILT(cur_t) # request SAMPEX count rate data
    h.load()
    
    # find index of where SAMPEX data time matches timestamp
    m = np.where(h.times == cur_t)[0][0]
    d_plot = h.times[m-b1:m+b2]
    r_plot = h.counts[m-b1:m+b2]
    
    maxdis = max(r_plot) - min(r_plot) # range of interval
    # identify peaks in interval
    [pks, _] = find_peaks(r_plot, prominence=0.25*maxdis, distance=3)
    if j in [0]:
        # The actual process of peaks found is a lot more finicky than this,
        # and can be found in the ChorusWaves search script. This version runs
        # a lot faster, albeit less accurately, so this line just fixes any discrepancy
        # between this method and the more robust way.
        pks = pks[0:4]
    elif j in [2]:
        pks = pks[2:6]
    elif j in [4]:
        pks = pks[1:5]
    
    # for plotting
    txt_loc_x = h.times[m-b1]
    txt_loc_y = 0.8*(max(r_plot) - min(r_plot)) + min(r_plot)
    xlbel = "Time (UTC) on " + cur_t.strftime('%m/%d/%Y')
    
    ax.plot(d_plot, r_plot)
    ax.plot(d_plot[pks], r_plot[pks], 'r*', markersize=10)
    ax.xaxis.set_major_formatter(dates.DateFormatter('%H:%M:%S')) 
    ax.set_xlabel(xlbel, fontsize=16)
    ax.grid()
    ax.set_ylabel("Count rate (#/20ms)", fontsize=16)
    ax.text(x=txt_loc_x, y=txt_loc_y, s=plotting_data["letter"][j], fontsize=20,
           backgroundcolor="white")
    ax.tick_params(labelsize=12)
    ax.set_ylim((min(r_plot)-0.1*np.ptp(r_plot)), (1.1*np.ptp(r_plot)+min(r_plot)))
    
# read in data
data = pd.read_csv("all_events_v2.csv", index_col=0)
good_data = data[data.final_eye<2] # restrict to good events only
good_data = good_data.reset_index(drop=True)

idx =  [15, 25, 12, 33, 64] # indices of events I picked as examples
plotting_data = {"t": [good_data.t[x] for x in idx], 
                 "letter": ['a) - decreasing', 'b) - crown', 'c) - flat', 'd) - half',
                            'f) - other']}

fig, ax = plt.subplots(2,3, figsize=(20,8), constrained_layout=True)
ax_flat = ax.flatten()
ax_flat[4].set_visible(False)

# iterate through axes object and plot each example event
for j, a in enumerate(ax_flat):
    if j not in [4, 5]:
        plot_date(a, j)
        if j == 3:
            a.set_position([0.15, -.05, 0.3, 0.5])
    elif j in [5]:
        plot_date(a, j-1)
        a.set_position([0.55, -0.05, 0.3, 0.5])


In [ ]:
'''
MLT & L-shell histograms
'''

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime

plt.rcParams["font.family"] = "Arial"

# read in data
data = pd.read_csv("all_events_v3.csv") # my data
ref = pd.read_csv("microburst_catalog_00.txt", sep=",") # reference data
pll = pd.read_csv("pll.csv", index_col=0) # portion in losscone 

# limit reference data to time surveyed by my search script
time_needed = pd.to_datetime(ref['dateTime'])
idx = np.where((time_needed >= datetime(2000, 1, 1)) & (time_needed <= datetime(2003, 12, 31)))[0]
ref = ref.iloc[idx,:]


def make_hist(ax1, param, rmin, rmax, bin_num, losscone):


    ax2 = ax1.twinx() # make second axis for reference data

    if losscone == "losscone":
        term = (pll.portion_losscone_1 < 1)
    else:
        term = (pll.portion_losscone_1 == 1)
    ax1.hist(data[param][(data.final_eye<2) & term], 
             bins=bin_num, histtype='step', range=(rmin,rmax),
             label='good bouncing packets',  color='black', linewidth=3, zorder=50)
    ax2.hist(ref[param], bins=bin_num, range=(rmin, rmax), histtype='step', label='all microbursts', 
             color='red', linewidth=3, zorder=20, linestyle='--') # plot reference data

    # tick/label stuff (formatting)
    ax1.tick_params(labelsize=12)
    ax1.set_xlabel(param, fontsize=14)
    ax1.set_ylabel('Counts (#), good bouncing packets', fontsize=14)
    ax1.grid()
    ax1.set_xlim(rmin, rmax)

    ax2.set_ylabel('Counts (#), all microbursts', color="lightcoral", fontsize=14)
    ax2.tick_params(axis='y', labelcolor="lightcoral", labelsize=12)
    ax2.set_xlim(rmin, rmax)

    
fig, ax = plt.subplots(2, 1, figsize=(14,11))
# # Call function for each variable
make_hist(ax[0], "MLT", 0, 24, 24, "losscone")
make_hist(ax[1], "L", 1, 8, 14, "losscone")

fig1, ax1 = plt.subplots(2, 1, figsize=(14,11))
# # Call function for each variable
make_hist(ax1[0], "MLT", 0, 24, 24, "no")
make_hist(ax1[1], "L", 1, 8, 14, "no")



In [ ]:
'''
MLT & L-shell histograms
'''

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime

plt.rcParams["font.family"] = "Arial"

# read in data
data = pd.read_csv("all_events_v3.csv") # my data
ref = pd.read_csv("microburst_catalog_00.txt", sep=",") # reference data

# limit reference data to time surveyed by my search script
time_needed = pd.to_datetime(ref['dateTime'])
idx = np.where((time_needed >= datetime(2000, 1, 1)) & (time_needed <= datetime(2003, 12, 31)))[0]
ref = ref.iloc[idx,:]

fig, ax = plt.subplots(2, figsize=(11, 18))
def make_hist(ax1, param, rmin, rmax, bin_num, shape):
    # Function that makes histograms of variables of your choosing
    # Author: Max Feinland
    # Date created: 7/20/2024
    # Last modified: 10/24/2024
    
    # Inputs: axes object (which subplot to plot on), variable name, 
    # minimum range, maximum range, number of bins
    
    # Outputs: none
    
    ax2 = ax1.twinx() # make second axis for reference data
    
    # plot the events
#     ax1.hist(data[param][(data.final_eye<2) & ((data.shapes=='decr') | (data.shapes=='crown') |
#                                               (data.shapes=='other'))], 
#              bins=bin_num, histtype='step', range=(rmin,rmax),
#              label='good bouncing packets',  color='black', linewidth=3, zorder=50)
    if shape == "y":
    ax1.hist(data[param][(data.final_eye<2) & (data.shapes==shape)], 
             bins=bin_num, histtype='step', range=(rmin,rmax),
             label='good bouncing packets',  color='black', linewidth=3, zorder=50)
    ax2.hist(ref[param], bins=bin_num, range=(rmin, rmax), histtype='step', label='all microbursts', 
             color='red', linewidth=3, zorder=20, linestyle='--') # plot reference data
    
    # tick/label stuff (formatting)
    ax1.tick_params(labelsize=12)
    ax1.set_xlabel(param, fontsize=14)
    ax1.set_ylabel('Counts (#), good bouncing packets', fontsize=14)
    ax1.grid()
    ax1.set_xlim(rmin, rmax)
    
    ax2.set_ylabel('Counts (#), all microbursts', color="lightcoral", fontsize=14)
    ax2.tick_params(axis='y', labelcolor="lightcoral", labelsize=12)
    ax2.set_xlim(rmin, rmax)
    
    # adding subplot label
    if param=="MLT":
        ax1.text(20, 2, shape, fontsize=20)
        ax1.set_xlabel("Magnetic Local Time", fontsize=14) # just because the variable is named
        # "MLT" in the datasets, but I wanted the x-axis label to be more descriptive
#     else:
#         ax1.text(1.25, 40, "b)", fontsize=20)
#         ax1.set_xlabel("L-shell", fontsize=14) # just because the variable is named
#         # "L" in the datasets, but I wanted the x-axis label to be more descriptive

# # # Call function for each variable
make_hist(ax[0], "MLT", 0, 24, 24, "y")
make_hist(ax[1], "MLT", 0, 24, 24, "n")
# make_hist(ax[2], "MLT", 0, 24, 24, "flat")
# make_hist(ax[3], "MLT", 0, 24, 24, "half")
# make_hist(ax[4], "MLT", 0, 24, 24, "losscone")
# make_hist(ax[5], "MLT", 0, 24, 24, "other")
# make_hist(ax[1], "L", 1, 8, 14)

In [ ]:
'''
Period vs. L-shell
'''

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.dates as dates
from matplotlib.legend_handler import HandlerTuple
import ast
import matplotlib.colors as mcolors

plt.rcParams["font.family"] = "Arial"

def plot_specific_shape(ax, k, fullper, norm, cmap, diff, pl1):
    # Function that plots bounce period vs. L shell for a specific shape
    # Author: Max Feinland
    # Date created: sometime in July 2024
    # Last modified: 8/30/2024
    
    # Inputs: axes object (which subplot to plot on), iterator variable
    
    # Outputs: none
    
    current_shape = txt["shapes"][k] # pull stuff out of dict
    current_letter = txt["letter"][k]
    
    # find indices containing allowable shapes
    indices = np.where(good_data.shapes==current_shape)[0]
    
    for j in indices:
        num_pks = len(s.dt[j])
        hilt_uncertainty = 0.02 # time resolution of instrument
        err = np.sqrt(good_data.spread[j]**2 + hilt_uncertainty**2) # propagate error

        full_diff = abs(s.dt[j] - good_data.tb[j])
        half_diff = abs(s.dt[j] - good_data.tb[j]/2)
        
        # are any of the spacings within allowable sigma? t/f (bool)
        full_yn = any([x <= err for x in full_diff])
        half_yn = any([x <= err for x in half_diff])
        
#         print(s.dt[j], err, full_diff, full_yn, half_diff, half_yn)
        
        # determine if half, full, or no match
        if full_yn & half_yn:
            # Both conditions are true, so figure out which is closer
            if min(full_diff) < min(half_diff):
                fullper[j] = 1
                edgecol = full_color
                tb = good_data.tb[j]
                zo = 10
                diff[j] = min(full_diff)
            else:
                fullper[j] = 2
                edgecol = half_color
                tb = good_data.tb[j]/2
                zo = 10
                diff[j] = min(half_diff)
        elif full_yn:
            fullper[j] = 1
            edgecol = full_color
            tb = good_data.tb[j]
            zo = 10
            diff[j] = min(full_diff)
        elif half_yn:
            fullper[j] = 2
            edgecol = half_color
            tb = good_data.tb[j]/2
            zo = 10
            diff[j] = min(half_diff)
        else:
            fullper[j] = 0
            edgecol = else_color
            zo = 5
            if min(full_diff) < min(half_diff):
                tb = good_data.tb[j]
                diff[j] = min(full_diff)
            else:
                tb = good_data.tb[j]/2
                diff[j] = min(half_diff)
                
        if pl1[j] == 1:
            marker = '^'
        else:
            marker = 'o'
        # plotting
        handl = ax.scatter(good_data.L[j]*np.ones(len(s.dt[j])), s.dt[j], marker=marker, s=100, 
                   c=pl1[j]*np.ones(len(s.dt[j])), cmap=cmap, norm=norm, alpha = 0.8, zorder=zo,
                          edgecolor='k')
#         ax.plot(good_data.L[j], tb, 's', markerfacecolor='none', markeredgecolor=edgecol,
#                markersize=10)
        ax.errorbar(good_data.L[j], tb, yerr=err, fmt='s', 
                 markerfacecolor='none', capsize=2.5, markeredgecolor=edgecol,
                 zorder=(zo+5), markersize=10, color='black', markeredgewidth=2, alpha=0.8)
    
    
    if current_shape == 'decr':
        shape_label = 'decreasing'
    else:
        shape_label = current_shape
        
    all_label = current_letter + ' - ' + shape_label
    ax.text(2, 0.9, all_label, fontsize=25) # subplot label
#     ax.text(5, 0.9, shape_label, fontsize=25)
    ax.grid()
    
    if k in [3, 4, 5]: # put xlabel on bottom 4 plots
        ax.set_xlabel("L-shell", fontsize=18)
    if k in [0, 3]: # put ylabel on leftmost 2 plots
        ax.set_ylabel("Mean period (s)", fontsize=18)
    ax.set_ylim(0, 1)
    ax.set_xlim(1.5, 7)
    ax.xaxis.set_tick_params(labelsize=15)
    ax.yaxis.set_tick_params(labelsize=15)
    return handl

# Import data
data = pd.read_csv("all_events_v3.csv", index_col=0)
good_data = data[data.final_eye<2] # restrict to good events
good_data = good_data.reset_index(drop=True) # reset index

# spread in model predictions
pers = pd.read_csv("model_pers_2.csv",index_col=0)
spread = pd.DataFrame({'spread': (pers.max(axis=1) - pers.min(axis=1))})
good_data = good_data.join(spread)

# spacings between peaks for each event
s = pd.read_csv("spacings.csv")
s['dt'] = s['dt'].apply(ast.literal_eval)
newper = [np.min(x) for x in s.dt] # data.per is min per, use this instead

# portion of particles in loss cone
pll = pd.read_csv("pll.csv", index_col=0)
pl1 = pll.portion_losscone_1
pl2 = pll.portion_losscone_2

# Shared color normalization and colormap
norm = mcolors.Normalize(vmin=0, vmax=1)
cmap = plt.cm.viridis  # Use any colormap of your choice

# Making figures
fig = plt.figure(layout='constrained', figsize=(20,18))
subfigs = fig.subfigures(2, 1, wspace=0.07)

## First figure: bounce period histograms
ax1 = subfigs[0].subplots()

# Calculate period ratios for each type 
good_ratio = np.divide(newper, good_data.tb)
# ok_ratio = np.divide(data.per[(data.final_eye>1.5) & (data.final_eye<3)], 
#                      data.tb[(data.final_eye>1.5) & (data.final_eye<3)])
bad_ratio = np.divide(data.per[data.final_eye>1.5], data.tb[data.final_eye>1.5])

ax1.hist(good_ratio, bins=15, range=(-0.05,1.45), color='royalblue', histtype='step', linewidth=4,
        linestyle='-', zorder=10, label='good') # good histogram
# ax1.hist(ok_ratio, bins=15, range=(-0.05,1.45), color='goldenrod', histtype='step', linewidth=4,
#          linestyle='-.', zorder=5, label='okay') # ok histogram
ax1.hist(bad_ratio, bins=15, range=(-0.05,1.45), color='dimgrey', histtype='step', linewidth=4,
         linestyle='--', zorder=0, label='bad') # bad histogram

# axis labels & ticks
ax1.set_xlabel("Ratio of observed period to predicted period", fontsize=20)
ax1.set_ylabel("Number of observations", fontsize=20)
ax1.xaxis.set_tick_params(labelsize=20)
ax1.yaxis.set_tick_params(labelsize=20)
ax1.legend(fontsize=20)
ax1.grid()
ax1.set_xlim(0, 1.5)
ax1.set_ylim(0, 80)
ax1.text(0.05, 75, 'a)', fontsize=30) # subplot label

# Second figure: bounce period vs. L for each shape
ax2 = subfigs[1].subplots(2, 3, sharex=True, sharey=True)
ax_flat = ax2.flatten()
ax_flat[4].set_visible(False)

txt = {"shapes": ["decr", "crown", "flat", "half", "other"],
      "letter": ["b)", "c)", "d)", "e)", "f)"]}

full_color = 'dodgerblue'
half_color = 'limegreen'
else_color = 'rosybrown'

fullper = np.zeros(len(good_data)) # initialize vector containing full/half classification
diff = np.zeros(len(good_data)) # initialize vector containing difference bw expected & observed
norm = mcolors.Normalize(vmin=0, vmax=1)
# iterate through axes object and plot each shape type
for j, a in enumerate(ax_flat):
    if j < 4:
        handle = plot_specific_shape(a, j, fullper, norm, cmap, diff, pll.portion_losscone_1)
        if j == 3:
            a.set_position([0.1, -.05, 0.45, 0.5])
    elif j == 5:
        a.set_position([0.6, -0.05, 0.45, 0.5])
        handle = plot_specific_shape(a, j-1, fullper, norm, cmap, diff, pll.portion_losscone_1)
cbar = fig.colorbar(handle, ax=ax_flat, orientation='vertical')
cbar.set_label("Portion in losscone if isotropic", fontsize=16)


p = ax2[0,2]
# p1, = p.plot(0, 0, 'o', color=full_color, alpha=0.8, markersize=10)
# p2, = p.plot(0, 0, 'o', color=half_color, alpha=0.8, markersize=10)
# p3, = p.plot(0, 0, 'o', color=else_color, alpha=0.8, markersize=10)
f = p.errorbar(0, 0, yerr=0.2, fmt='s', markeredgecolor=full_color, markerfacecolor='none',
               color='black', markersize=10, capsize=4, markeredgewidth=2)
h = p.errorbar(0, 0, yerr=0.2, fmt='s', markeredgecolor=half_color, markerfacecolor='none',
               color='black', markersize=10, capsize=4, markeredgewidth=2)
x = p.errorbar(0, 0, yerr=0.2, fmt='s', markeredgecolor=else_color, markerfacecolor='none',
               color='black', markersize=10, capsize=4, markeredgewidth=2)
# l = p.legend([(p1, p2, p3), f, h, x], # i made the legend look nice
#          ['Observed spacing', 'T05 model', 'T05 model (half)', 'T05 model (no match)'], 
#          handler_map={tuple: HandlerTuple(ndivide=None)}, fontsize=16) 
l = p.legend([handle, f, h, x], # i made the legend look nice
         ['Observed spacing', 'T05 model', 'T05 model (half)', 'T05 model (no match)'], 
             fontsize=16) 

In [ ]:
'''Make polar plot of microburst distribution and compare to Hamdans paper'''

In [ ]:
'''Geographical location'''
def figure_4():

    import numpy as np
    import pandas as pd
    import geopandas as gpd
    import matplotlib.pyplot as plt
    from matplotlib.colors import LinearSegmentedColormap
    import ast

    plt.rcParams["font.family"] = "Arial" # because I don't like the default Python font
    
    def determine_match(full_diff, half_diff, full_yn, half_yn):  
        if full_yn and half_yn: # if both half & full-per matches
            if min(full_diff) < min(half_diff):
                return 1
            else:
                return 2
        elif full_yn: # if full per match only
            return 1
        elif half_yn: # if half per match only
            return 2
        else: # no match
            return 0


    def make_losscone_map(ax):
        # Function that makes world outline and L-shell contours
        # Inputs: axes object (which subplot to plot on)

        world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
        world.plot(ax=ax, color='none', edgecolor='black', zorder=40)

        # read in SAMPEX attitude data. If you don't have this file you can download it from here:
        # https://izw1.caltech.edu/sampex/DataCenter/DATA/PSSet/Text/PSSet_6sec_2000022_2000048.txt
        # Make sure to change the pathing so it works with your machine.
        att = pd.read_csv('C:/Users/maxim/sampex-data/Attitude/PSSet_6sec_2000022_2000048.txt', 
                         sep=' ', header=60, on_bad_lines='skip')

        cols = np.array([7, 8, 35]) # take out lat, lon, losscone 2
        att = att.iloc[:150000, cols] # don't need the whole thing, just about 14 days for full coverage
        att.columns = ['lon', 'lat', 'losscone']

        # change longitude to be -180 to 180
        long_idx = np.where(att.lon > 180)[0]
        att.lon[long_idx] = att.lon[long_idx] - 360

        sc = ax.scatter(att.lon, att.lat, c=att.losscone, s=25, vmin=30, vmax=90, zorder=5, cmap='gray')
        c = plt.colorbar(sc)
        c.set_label("Losscone angle (deg)", fontsize=16)

        lgrid = pd.read_csv('Lgrid.dat', delimiter='\t', header=None)

        for i in np.arange(1, 30, 2):
            min_pos = np.argmin(lgrid.iloc[:,i])
            latl = np.concatenate(([lgrid.iloc[min_pos:,i-1], lgrid.iloc[:min_pos,i-1]]))
            lonl = np.concatenate(([lgrid.iloc[min_pos:,i], lgrid.iloc[:min_pos,i]]))
            ax.plot(lonl, latl, '--', color="white", zorder=45)

        ax.text(-180, 7, "Magnetic Equator", fontsize=16, zorder=50)
        ax.text(-197, 48, "L = 2", fontsize=14, zorder=50)
        ax.text(-197, -40, "L = 2", fontsize=14, zorder=50)
        ax.text(-197, 59, "L = 3", fontsize=14, zorder=50)
        ax.text(-197, -55, "L = 3", fontsize=14, zorder=50)
        ax.text(-197, 64, "L = 5", fontsize=14, zorder=50)
        ax.text(-197, -60, "L = 5", fontsize=14, zorder=50)
        ax.text(-197, 71, "L = 8", fontsize=14, zorder=50)
        ax.text(-197, -66, "L = 8", fontsize=14, zorder=50)

        ax.set_xlabel("Longitude", fontsize=20)
        ax.set_ylabel("Latitude", fontsize=20)
        ax.set_ylim(-85, 85)
        return ax

    # Import data
    data = pd.read_csv("all_events_v2.csv", index_col=0)
    good_data = data[data.final_eye<2] # restrict to good events
    good_data = good_data.reset_index(drop=True) # reset index

    # spread in model predictions
    pers = pd.read_csv("model_preds_and_spacing.csv",index_col=0)
    models  = pers[['T89', 'T05', 'OP', 'SL']]
    pers['spread'] = models.max(axis=1) - models.min(axis=1)

    # spacings between peaks for each event
    pers['dt'] = pers['dt'].apply(ast.literal_eval)
    newper = [np.mean(x) for x in pers.dt] # calculate new mean spacing


    fig, [ax1, ax2] = plt.subplots(2, 1, figsize=(18, 14))

    # call losscone map function
    ax1 = make_losscone_map(ax1)
    ax2 = make_losscone_map(ax2)

    fullper = np.zeros(len(good_data)) # initialize vector containing full/half classification

    # create full/half classification
    # this is kept dynamic so you can change the error condition if you need to
    for j in range(len(good_data.tb)):
            hilt_uncertainty = 0.02 # time resolution of instrument
            err = np.sqrt(pers.spread[j]**2 + hilt_uncertainty**2) # propagate error

            fulldiff = abs(pers.dt[j] - good_data.tb[j])
            halfdiff = abs(pers.dt[j] - good_data.tb[j]/2)

            # are any of the spacings within allowable error bounds? 
            fullyn = any([x <= err for x in fulldiff])
            halfyn = any([x <= err for x in halfdiff])
            
            fullper[j] = determine_match(fulldiff, halfdiff, fullyn, halfyn)

    # subplot 1: half/full/fail
    full = ax1.scatter(good_data.lon[fullper==1], good_data.lat[fullper==1], 
                      s=100,  c='dodgerblue', label='full period', zorder=55, edgecolor='navy')
    half = ax1.scatter(good_data.lon[fullper==2], good_data.lat[fullper==2], 
                      s=100,  c='limegreen', label='half period', zorder=50, edgecolor='darkgreen')
    fail = ax1.scatter(good_data.lon[fullper==0], good_data.lat[fullper==0], 
                      s=100,  c='rosybrown', label='no match', zorder=45, edgecolor='maroon')
    ax1.legend(loc='right', fontsize=14).set_zorder(50)

    # subplot 2: shapes
    decr = ax2.scatter(good_data.lon[good_data.shapes=="decr"], good_data.lat[good_data.shapes=="decr"], 
                      s=100,  c='royalblue', label='decreasing', zorder=50, edgecolor='navy')
    blake = ax2.scatter(good_data.lon[good_data.shapes=="blake"], good_data.lat[good_data.shapes=="blake"],
                       s=100, c='mediumslateblue',  label='blake', zorder=50, edgecolor='rebeccapurple')
    crown = ax2.scatter(good_data.lon[good_data.shapes=="crown"], good_data.lat[good_data.shapes=="crown"],
                       s=100, c='fuchsia', label='crown', zorder=50, edgecolor='purple')
    flat = ax2.scatter(good_data.lon[good_data.shapes=="flat"], good_data.lat[good_data.shapes=="flat"],
                       s=100, c='turquoise', label='flat', zorder=50, edgecolor='darkslategray')
    incr = ax2.scatter(good_data.lon[good_data.shapes=="incr"], good_data.lat[good_data.shapes=="incr"],
                       s=100, c='orangered', label='increasing', zorder=50, edgecolor='maroon')
    smile = ax2.scatter(good_data.lon[good_data.shapes=="smile"], good_data.lat[good_data.shapes=="smile"],
                       s=100, c='purple', label='smile', zorder=50, edgecolor='darkmagenta')
    half = ax2.scatter(good_data.lon[good_data.shapes=="half"], good_data.lat[good_data.shapes=="half"],
                       s=100, c='goldenrod', label='half', zorder=50, edgecolor='darkgoldenrod')
    other = ax2.scatter(good_data.lon[good_data.shapes=="other"], good_data.lat[good_data.shapes=="other"],
                       s=100, c='lawngreen', label='other', zorder=50, edgecolor='olivedrab')

    ax2.legend(loc='right', fontsize=14).set_zorder(50)
figure_4()

In [ ]:
def isleap(year):
    return year % 4 == 0

def dayarrayfun():
    """Generate a DataFrame of day and year values."""
    start_year = 1996
    start_day = 160
    end_year = 2012
    end_day = 283
    spacing = 27

    days_array = []
    years_array = []

    current_year = start_year
    current_day = start_day

    while current_year < end_year or (current_year == end_year and current_day <= end_day):
        # Determine the number of days in the current year
        num_days = 366 if isleap(current_year) else 365
        
        # Append the current day and year to the arrays
        days_array.append(current_day)
        years_array.append(current_year)
        
        # Increment the current day by the spacing
        current_day += spacing
        if current_day > num_days:
            current_day -= num_days
            current_year += 1

    y = [str(x) + str(y) for x, y in zip(years_array, days_array)]
    dates = pd.to_datetime(y, format='%Y%j')
    days_df = pd.DataFrame({'dates': dates, 'y_col': years_array, 'd_col': days_array})
    return days_df

In [ ]:
def load_att(t):
    days_df = dayarrayfun()
    pathname = 'C:/Users/maxim/sampex-data/Attitude/'
    # find the right row in the ephemeris files
    att_row = np.where(days_df['dates'] < t)[0][-1]
    day1str = str(days_df['d_col'][att_row])
    day2str = str(days_df['d_col'][att_row+1]-1)
    
    # checking if leading zeroes are needed
    if len(day1str) == 1:
        day1str = '00' + day1str
    elif len(day1str) == 2:
        day1str = '0' + day1str

    if len(day2str) == 1:
        day2str = '00' + day2str
    elif len(day2str) == 2:
        day2str = '0' + day2str
    filename = 'PSSet_6sec_' + str(days_df['y_col'][att_row]) + day1str + '_' + \
                str(days_df['y_col'][att_row+1]) + day2str + '.txt'
    try:
        att = pd.read_csv(pathname + filename, sep=' ', header=60, on_bad_lines='skip')
    except:
        a = sampex.Attitude(t) # download da file
        att = pd.read_csv(pathname + filename, sep=' ', header=60, on_bad_lines='skip')

    cols = np.array([0, 1, 2, 7, 8, 34, 35, 68]) # take out lat, lon, losscone 2
    att = att.iloc[:, cols] 
    att.columns = ['year', 'doy', 'second', 'lon', 'lat', 'losscone1', 'losscone2', 'pitch']

    # change longitude to be -180 to 180
    long_idx = np.where(att.lon > 180)[0]
    att.lon[long_idx] = att.lon[long_idx] - 360

    year = np.array(att.iloc[:, 0])
    doy = np.array(att.iloc[:, 1])
    second = np.array(att.iloc[:, 2])
    
    def calc_t(year, doy, s):
        start_of_year = datetime(int(year), 1, 1)
        return start_of_year + timedelta(days=int(doy) - 1, seconds=int(s))

    vectorized_datetime = np.vectorize(calc_t)
    att_t = vectorized_datetime(year, doy, second)
    return att_t, att

def interpolate_pitch_and_losscone(att_t, att, a_idx, t):
    l = []
    t1 = att_t[a_idx-1] # time 1 to interpolate with
    t2 = att_t[a_idx] # time 2, after desired timestamp
    dt = (t2 - t1).total_seconds() # change in time
    att_keys = ['pitch', 'losscone1', 'losscone2'] # values you want to interpolate
    for k in att_keys:
        dy = att[k][a_idx] - att[k][a_idx-1] # change in y for whichever key you're on
        slope = dy/dt # change in y over change in time
        fval = att[k][a_idx-1] + slope * (t - t1).total_seconds() # interpolation at time t
        l.append(fval)
    return l

In [ ]:
'''Comparing pitch angle w/ losscone angle'''
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
# from IRBEM import MagFields
att_already_loaded = None
print_yn = True
x = []

days_df = dayarrayfun() # call list of year-doy pairs

# Import data
data = pd.read_csv("all_events_v2.csv", index_col=0)
good_data = data[data.final_eye<2] # restrict to good events
good_data = good_data.reset_index(drop=True) # reset index

for i in range(len(data)):

    t = datetime.strptime(data['t'][i], '%Y-%m-%d %H:%M:%S.%f') # convert from string to datetime
    att_row = np.where(days_df['dates'] < t)[0][-1] 
    
    # Check to see if attitude loading needed
    if 'old_row' in globals():
        if att_row != old_row:
            att_already_loaded = False
        else:
            att_already_loaded = True
        
    # Load attitude data if needed
    if att_already_loaded is not True:
        if print_yn == True:
            print(f"\nLoading attitude data...")
        att_t, att = load_att(t)
        old_row = att_row

    a_idx = np.where(att_t > t)[0][0] # first time attitude time data is after desired timestamp
    # Interpolate values
    pll = interpolate_pitch_and_losscone(att_t, att, a_idx, t) # pitch, losscone1, losscone2
    x.append(pll)
    print(x)
    
p = []
l1 = []
l2 = []
for i in range(len(x)):
    p.append(x[i][0])
    l1.append(x[i][1])
    l2.append(x[i][2])


# LLA = {'x1':att.iloc[row,9], 'x2':att.iloc[row,7], 'x3':att.iloc[row,8], 'dateTime':datestring}
# maginput = {'Kp':40}
# output_dictionary = model.make_lstar_shell_splitting(LLA, alpha, maginput)
# print(output_dictionary)

In [ ]:
'''Geographical location'''
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import ast

plt.rcParams["font.family"] = "Arial" # because I don't like the default Python font

fig, ax = plt.subplots(figsize=(18, 14))
ax.set_facecolor('deepskyblue')
shapefile = 'ne_110m_admin_0_countries_lakes.shp'

world = gpd.read_file(shapefile)
world.plot(ax=ax, color='springgreen', edgecolor='black', zorder=40)

att = pd.read_csv('C:/Users/maxim/sampex-data/Attitude/PSSet_6sec_2000022_2000048.txt', 
                 sep=' ', header=60, on_bad_lines='skip')

cols = np.array([7, 8, 35]) # take out lat, lon, losscone 2
att = att.iloc[:150000, cols] # don't need the whole thing, just about 14 days for full coverage
att.columns = ['lon', 'lat', 'losscone']


lgrid = pd.read_csv('Lgrid.dat', delimiter='\t', header=None)

for i in np.arange(1, 30, 2):
    min_pos = np.argmin(lgrid.iloc[:,i])
    latl = np.concatenate(([lgrid.iloc[min_pos:,i-1], lgrid.iloc[:min_pos,i-1]]))
    lonl = np.concatenate(([lgrid.iloc[min_pos:,i], lgrid.iloc[:min_pos,i]]))
    ax.plot(lonl, latl, '--', color="white", zorder=45)

ax.text(-180, 7, "Magnetic Equator", fontsize=16, zorder=50)
ax.text(-197, 48, "L = 2", fontsize=14, zorder=50)
ax.text(-197, -40, "L = 2", fontsize=14, zorder=50)
ax.text(-197, 59, "L = 3", fontsize=14, zorder=50)
ax.text(-197, -55, "L = 3", fontsize=14, zorder=50)
ax.text(-197, 64, "L = 5", fontsize=14, zorder=50)
ax.text(-197, -60, "L = 5", fontsize=14, zorder=50)
ax.text(-197, 71, "L = 8", fontsize=14, zorder=50)
ax.text(-197, -66, "L = 8", fontsize=14, zorder=50)

ax.set_xlabel("Longitude", fontsize=20)
ax.set_ylabel("Latitude", fontsize=20)
ax.set_ylim(-85, 85)

# Import data
data = pd.read_csv("all_events_v2.csv", index_col=0)
good_data = data[data.final_eye<2] # restrict to good events
good_data = good_data.reset_index(drop=True) # reset index

ax.plot(good_data.lon[0], good_data.lat[0], '*', markersize=20, color='red', zorder=50)
print(pll_df.p[0], pll_df.l1[0], pll_df.l2[0])

In [ ]:
# really_good_days = np.where(good_data.final_eye==1)[0]
pll_df = pd.read_csv('pll.csv', index_col=0)

for i in range(len(data)):
    f, a = plt.subplots(figsize=(7,7))

    # coordinate axes
    a.plot(np.linspace(-1.5, 1.5, 20), np.zeros(20), 'k-', zorder=0)
    a.plot(np.zeros(20), np.linspace(-1.5, 1.5, 20), 'k-', zorder=0)

    # pitch angle
    a.quiver(0, 0, np.cos(np.deg2rad(90-pll_df.p[i])), np.sin(np.deg2rad(90-pll_df.p[i])), 
             angles='xy', scale_units='xy', scale=1, color='r', label='Pitch')
    print('pitch angle:', pll_df.p[i])

    # either side due to FOV
    a.quiver(0, 0, np.cos(np.deg2rad(90-pll_df.p[i] + 34)), np.sin(np.deg2rad(90-pll_df.p[i] + 34)), 
             angles='xy', scale_units='xy', scale=1, color='salmon', label='Pitch at edge of FOV', 
             width= 0.002)
    a.quiver(0, 0, np.cos(np.deg2rad(90-pll_df.p[i] - 34)), np.sin(np.deg2rad(90-pll_df.p[i] - 34)), 
             angles='xy', scale_units='xy', scale=1, color='salmon', label=None, width= 0.002)

    # loss cone 1
    a.quiver(0, 0, np.cos(np.deg2rad(90-pll_df.l1[i])), np.sin(np.deg2rad(90-pll_df.l1[i])), 
             angles='xy', scale_units='xy', scale=1, color='orange', 
             label='Loss cone (same hemisphere)')
    a.quiver(0, 0, -np.cos(np.deg2rad(90-pll_df.l1[i])), np.sin(np.deg2rad(90-pll_df.l1[i])), 
             angles='xy', scale_units='xy', scale=1, color='orange')
    # southern hemisphere
    a.quiver(0, 0, np.cos(np.deg2rad(90-pll_df.l1[i])), -np.sin(np.deg2rad(90-pll_df.l1[i])), 
             angles='xy', scale_units='xy', scale=1, color='orange')
    a.quiver(0, 0, -np.cos(np.deg2rad(90-pll_df.l1[i])), -np.sin(np.deg2rad(90-pll_df.l1[i])), 
             angles='xy', scale_units='xy', scale=1, color='orange')
    
    # loss cone 2
    a.quiver(0, 0, np.cos(np.deg2rad(90-pll_df.l1[i])), np.sin(np.deg2rad(90-pll_df.l2[i])), 
             angles='xy', scale_units='xy', scale=1, color='gold', 
             label='Loss cone (either hemisphere)')
    a.quiver(0, 0, -np.cos(np.deg2rad(90-pll_df.l1[i])), np.sin(np.deg2rad(90-pll_df.l2[i])), 
             angles='xy', scale_units='xy', scale=1, color='gold')
    # southern hemisphere
    a.quiver(0, 0, np.cos(np.deg2rad(90-pll_df.l1[i])), -np.sin(np.deg2rad(90-pll_df.l2[i])), 
             angles='xy', scale_units='xy', scale=1, color='gold')
    a.quiver(0, 0, -np.cos(np.deg2rad(90-pll_df.l1[i])), -np.sin(np.deg2rad(90-pll_df.l2[i])), 
             angles='xy', scale_units='xy', scale=1, color='gold')
    
    # magnetic field
    a.quiver(0, 0, 0, 1, angles='xy', scale_units='xy', scale=1, color='b', label='-B-field')
    a.quiver(0, 0, 0, -1, angles='xy', scale_units='xy', scale=1, color='navy')
    
    a.text(-1.45, 1.4, str(good_data.t[i]))
    a.text(-1.45, 1.3, 'Pitch angle: ' + str(round(pll_df.p[i], 2)) + '$^{\circ}$')
    a.text(-1.45, 1.2, 'Loss cone 1: ' + str(round(pll_df.l1[i],2)) + '$^{\circ}$')
    a.text(-1.45, 1.1, 'Loss cone 2: ' + str(round(pll_df.l2[i], 2)) + '$^{\circ}$')

    a.set_xlim(-1.5, 1.5)
    a.set_ylim(-1.5,1.5)
    a.legend()
    plt.show()

In [ ]:
'''Determine each % of particles observed within loss cone.
Mathematically, I think I can just say that the angle is 1/360th of the total, right?'''
# So, the portion of particles in the loss cone is the angle from the outermost edge of pitch at 
# edge of FOV to the 
# over 68deg always.
# prop1 = []
prop2 = []
for i in range(len(pll_df)):
#     print(i, good_data.t[i])
    pitch_angle = pll_df.p[i]
#     loss_cone_1 = pll_df.l1[i]
    loss_cone_1 = pll_df.l2[i]
    pitch_upper_limit = pitch_angle + 34
    pitch_lower_limit = pitch_angle - 34
#     print('pitch_upper_limit', round(pitch_upper_limit,2), 'pitch_lower_limit', 
#           round(pitch_lower_limit, 2), 'loss_cone_1', round(loss_cone_1,2))

    if loss_cone_1 == 90:
        portion_in_loss_cone1 = 1 #everything in loss cone
    elif (pitch_lower_limit < 90) and (pitch_upper_limit < (180-loss_cone_1)) :
        if (pitch_lower_limit < 0) and (pitch_upper_limit < loss_cone_1):
            portion_in_loss_cone1 = 1 #everything in loss cone
        elif loss_cone_1 > pitch_lower_limit:
            portion_in_loss_cone1 = (loss_cone_1 - pitch_lower_limit)/68
    elif (pitch_upper_limit > 90) and (pitch_lower_limit > loss_cone_1):
        if (pitch_upper_limit > (180-loss_cone_1)) and (pitch_lower_limit > (180-loss_cone_1)):
            portion_in_loss_cone1 = 1 #everything in loss cone
        else:
            portion_in_loss_cone1 = (pitch_upper_limit - (180-loss_cone_1))/68
    elif (pitch_lower_limit < 90) and (loss_cone_1 > pitch_lower_limit) and \
    (pitch_upper_limit > (180-loss_cone_1)):
        portion_in_loss_cone1 = ((pitch_upper_limit - (180-loss_cone_1)) + 
                                (loss_cone_1 - pitch_lower_limit))/68
    else:
#         print(' fuck')
        break
#     print(portion_in_loss_cone1)
    if (portion_in_loss_cone1 > 1) or (portion_in_loss_cone1 < 0):
#         print('bad')
        break
    prop2.append(portion_in_loss_cone1)
        

In [ ]:
df = pd.DataFrame({'portion_losscone_1': prop1, 'portion_losscone_2': prop2})

In [ ]:
pll_df = pd.read_csv('pll.csv', index_col=0)
final_df = pll_df.join(df)

In [ ]:
print(final_df)
final_df.to_csv('pll.csv')